In [0]:
%sql
USE CATALOG portfolio_catalog;
USE SCHEMA databricks_pipeline;
CREATE VOLUME IF NOT EXISTS portfolio_catalog.databricks_pipeline.mlflow_tmp;

In [0]:
%sql
SHOW VOLUMES


In [0]:
import pickle
import mlflow
import os
from pyspark.ml import PipelineModel
import pandas as pd

run_id = "7c0dde313ae64bd59f56218feec73a11"

os.environ["MLFLOW_DFS_TMP"] = "/Volumes/portfolio_catalog/databricks_pipeline/mlflow_tmp"

# Load model from UC volume
model_path = "/Volumes/portfolio_catalog/databricks_pipeline/data/gbt_model"
model = PipelineModel.load(model_path)

# Load signature + input example
with open("/Volumes/portfolio_catalog/databricks_pipeline/data/gbt_model/model_signature.pkl", "rb") as f:
    signature = pickle.load(f)

input_example = spark.createDataFrame(
    pd.read_parquet("/Volumes/portfolio_catalog/databricks_pipeline/data/gbt_model/input_example.parquet")
)

# mlflow.set_experiment(experiment_id="1862660362659971")



In [0]:

# with mlflow.start_run():
model_name = "favorita_multi_model"
# with mlflow.start_run(run_name=model_name, nested=True, run_id=run_id):
with mlflow.start_run(run_id=run_id):

    # mlflow.log_param("model_name", model_name)
    # mlflow.log_param("model_type", "global")

    mlflow.spark.log_model(
        spark_model=model,
        artifact_path="model",
        registered_model_name="gbt_forecast_model",
        signature=signature,
        input_example=input_example.toPandas(),
    )